In [1]:
import matplotlib.pyplot as plt
#from gradient_check import eval_numerical_gradient_array, eval_numerical_gradient
from layers import *
from fast_layers import *
from solver import Solver
%load_ext autoreload
%autoreload 2

## Extract data

In [6]:
from image_utils import *
import time

start_time = time.time()

Nsubjects = 6
print("Data for %d sujects:" % Nsubjects)

# Probe (validation)
X_val, y_val = get_data_sametaum(Nsubjects,"probe",1,1)
y_val = y_val - 1
print(X_val.shape)
print(y_val.shape)

# Gallery (train and test)
X_train, y_train = get_data_sametaum(Nsubjects, "gallery",1,1)
y_train = y_train - 1
print(X_train.shape)
print(y_train.shape)


print("--- %s seconds ---" % (time.time() - start_time))

Data for 6 sujects:


ValueError: attempt to get argmax of an empty sequence

## Train CNN
### Architecture: conv - relu - 2x2 max pool - affine - relu - affine - softmax
#### Using only tau=1 and m=1 recurrence plots

In [4]:
from cnn import ConvNet
from solver import *
import time
from image_utils import *


start_time = time.time()

# Prepare data

N = X_train.shape[0]
new_order = rand.permutation(N) #np.random:データフレームのランダムな並び替え

data2 = data = {
    'X_train': X_train,
    'y_train': y_train,
    'X_val': X_val,
    'y_val': y_val
  }

print("\nData shape:")
for k, v in data2.iteritems():
  print '%s: ' % k, v.shape
print("\n")


# Parameters

Number_of_filters = 32
Filter_size = 7
Hidden_dimension = 100
Weight_scale = 0.001   # np.sqrt(2.0/N) ?
Regularization_strength = 0.00001
Number_of_epochs = 20
Batch_size = 10
Learning_rate = 1e-4



# Train the network

start_time = time.time()

model = ConvNet(input_dim=(3, 128, 128),
                     num_filters = Number_of_filters,
                     filter_size = Filter_size,
                     hidden_dim = Hidden_dimension,
                     num_classes = Nsubjects,
                     weight_scale = Weight_scale,
                     reg = Regularization_strength,
                     dtype = np.float32) #np.float32

solver = Solver(model, data,
                num_epochs = Number_of_epochs,
                batch_size = Batch_size,
                update_rule = 'adam',
                optim_config = {'learning_rate': Learning_rate,},
                verbose=True, print_every=20)

solver.train()

t = time.time() - start_time
print("--- %s seconds ---" % t)



# Create report

comment="Only for tau=1 and m=1"
write_report(Nsubjects,
             Number_of_filters,
             Filter_size, 
             Hidden_dimension, 
             Weight_scale, 
             Regularization_strength, 
             Learning_rate, 
             Number_of_epochs,
             Batch_size, 
             solver.train_acc_history[-1], 
             solver.val_acc_history[-1],
             solver.loss_history[-1], 
             t, 
             comment)

plt.subplot(2, 1, 1)
plt.plot(solver.loss_history, 'o')
plt.xlabel('iteration')
plt.ylabel('loss')

plt.subplot(2, 1, 2)
plt.plot(solver.train_acc_history, '-o')
plt.plot(solver.val_acc_history, '-o')
plt.legend(['train', 'val'], loc='upper left')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.show()

# Plot validation accuracy per subject

plt.subplot(2, 1, 2)
plt.plot(solver.val_acc['subject_1'], '-o')
plt.plot(solver.val_acc['subject_2'], '-o')
plt.plot(solver.val_acc['subject_3'], '-o')
plt.plot(solver.val_acc['subject_4'], '-o')
plt.plot(solver.val_acc['subject_5'], '-o')
plt.plot(solver.val_acc['subject_6'], '-o')
plt.legend(['1', '2', '3', '4', '5', '6'], loc='upper right')
plt.xlabel('epoch')
plt.ylabel('val accuracy')
plt.show()


print("Training set:"),
unique, counts = np.unique(y_train, return_counts=True) #np.unique:重複をなくす
for i in range(unique.shape[0]):
    print '\t%d: %d' % (unique[i]+1, counts[i]),

print("\nValid set:"),
unique, counts = np.unique(y_val, return_counts=True) #np.unique
for i in range(unique.shape[0]):
    print '\t%d: %d' % (unique[i]+1, counts[i]),
print("\n")

NameError: name 'X_train' is not defined

### Using more regularization

In [33]:
from cnn import ConvNet
from solver import *
import time
from image_utils import *
import time


start_time = time.time()


# Prepare data

N = X_train.shape[0]
new_order = np.random.permutation(N)

data2 = data = {
    'X_train': X_train,
    'y_train': y_train,
    'X_val': X_val,
    'y_val': y_val
  }

print("\nData shape:")
for k, v in data2.iteritems():
  print '%s: ' % k, v.shape
print("\n")



# Parameters

Number_of_filters = 32
Filter_size = 7
Hidden_dimension = 100
Weight_scale = 0.01
Regularization_strength = 0.001
Number_of_epochs = 20
Batch_size = 10
Learning_rate = 1e-4



# Train the network

start_time = time.time()

model = ConvNet(input_dim=(3, 128, 128),
                     num_filters = Number_of_filters,
                     filter_size = Filter_size,
                     hidden_dim = Hidden_dimension,
                     num_classes = Nsubjects,
                     weight_scale = Weight_scale,
                     reg = Regularization_strength,
                     dtype = np.float32)

solver = Solver(model, data,
                num_epochs = Number_of_epochs,
                batch_size = Batch_size,
                update_rule = 'adam',
                optim_config = {'learning_rate': Learning_rate,},
                verbose=True, print_every=200)

solver.train()

t = time.time() - start_time
print("--- %s seconds ---" % t)



# Create report

comment="More regularization"
write_report(Nsubjects,
             Number_of_filters,
             Filter_size, 
             Hidden_dimension, 
             Weight_scale, 
             Regularization_strength, 
             Learning_rate, 
             Number_of_epochs,
             Batch_size, 
             solver.train_acc_history[-1], 
             solver.val_acc_history[-1],
             solver.loss_history[-1], 
             t, 
             comment)

plt.subplot(2, 1, 1)
plt.plot(solver.loss_history, 'o')
plt.xlabel('iteration')
plt.ylabel('loss')

plt.subplot(2, 1, 2)
plt.plot(solver.train_acc_history, '-o')
plt.plot(solver.val_acc_history, '-o')
plt.legend(['train', 'val'], loc='upper left')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.show()

# Plot validation accuracy per subject

plt.subplot(2, 1, 2)
plt.plot(solver.val_acc['subject_1'], '-o')
plt.plot(solver.val_acc['subject_2'], '-o')
plt.plot(solver.val_acc['subject_3'], '-o')
plt.plot(solver.val_acc['subject_4'], '-o')
plt.plot(solver.val_acc['subject_5'], '-o')
plt.plot(solver.val_acc['subject_6'], '-o')
plt.legend(['1', '2', '3', '4', '5', '6'], loc='upper right')
plt.xlabel('epoch')
plt.ylabel('val accuracy')
plt.show()

# Plot datasets compostion:

print("Training set:"),
unique, counts = np.unique(y_train, return_counts=True) #np.unique
for i in range(unique.shape[0]):
    print '\t%d: %d' % (unique[i]+1, counts[i]),

print("\nValid set:"),
unique, counts = np.unique(y_val, return_counts=True) #np.unique
for i in range(unique.shape[0]):
    print '\t%d: %d' % (unique[i]+1, counts[i]),
print("\n")

NameError: name 'X_train' is not defined

### Using dropout

In [34]:
from cnn import ConvNetDrop
from solver import *
import time
from image_utils import *
import time


start_time = time.time()


# Prepare data

N = X_train.shape[0]
new_order = np.random.permutation(N) #np.random

data2 = data = {
    'X_train': X_train,
    'y_train': y_train,
    'X_val': X_val,
    'y_val': y_val
  }

print("\nData shape:")
for k, v in data2.iteritems():
  print '%s: ' % k, v.shape
print("\n")



# Parameters

Number_of_filters = 32
Filter_size = 7
Hidden_dimension = 100
Weight_scale = 0.001   # np.sqrt(2.0/N) ?
Regularization_strength = 0.00001
Number_of_epochs = 20
Batch_size = 10
Learning_rate = 1e-5

dropout = 0.5
seed = 123



# Train the network

start_time = time.time()

model = ConvNetDrop(input_dim=(3, 128, 128),
                     num_filters = Number_of_filters,
                     filter_size = Filter_size,
                     hidden_dim = Hidden_dimension,
                     num_classes = Nsubjects,
                     weight_scale = Weight_scale,
                     reg = Regularization_strength,
                     dtype = np.float32, #np.float32
                     dropout = dropout,
                     seed = seed)

solver = Solver(model, data,
                num_epochs = Number_of_epochs,
                batch_size = Batch_size,
                update_rule = 'adam',
                optim_config = {'learning_rate': Learning_rate,},
                verbose=True, print_every=1000)

solver.train()

t = time.time() - start_time
print("--- %s seconds ---" % t)



# Create report

comment="Only for tau=1 and m=1, using DROPOUT=0.5"
write_report(Nsubjects,
             Number_of_filters,
             Filter_size, 
             Hidden_dimension, 
             Weight_scale, 
             Regularization_strength, 
             Learning_rate, 
             Number_of_epochs,
             Batch_size, 
             solver.train_acc_history[-1], 
             solver.val_acc_history[-1],
             solver.loss_history[-1], 
             t, 
             comment)

plt.subplot(2, 1, 1)
plt.plot(solver.loss_history, 'o')
plt.xlabel('iteration')
plt.ylabel('loss')

plt.subplot(2, 1, 2)
plt.plot(solver.train_acc_history, '-o')
plt.plot(solver.val_acc_history, '-o')
plt.legend(['train', 'val'], loc='upper left')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.show()

# Plot validation accuracy per subject

plt.subplot(2, 1, 2)
plt.plot(solver.val_acc['subject_1'], '-o')
plt.plot(solver.val_acc['subject_2'], '-o')
plt.plot(solver.val_acc['subject_3'], '-o')
plt.plot(solver.val_acc['subject_4'], '-o')
plt.plot(solver.val_acc['subject_5'], '-o')
plt.plot(solver.val_acc['subject_6'], '-o')
plt.legend(['1', '2', '3', '4', '5', '6'], loc='upper right')
plt.xlabel('epoch')
plt.ylabel('val accuracy')
plt.show()

# Plot datasets compostion:

print("Training set:"),
unique, counts = np.unique(y_train, return_counts=True) #np.unique
for i in range(unique.shape[0]):
    print '\t%d: %d' % (unique[i]+1, counts[i]),

print("\nValid set:"),
unique, counts = np.unique(y_val, return_counts=True) #np.unique
for i in range(unique.shape[0]):
    print '\t%d: %d' % (unique[i]+1, counts[i]),
print("\n")

NameError: name 'X_train' is not defined

### Try with Probes as training set and Gallery as validation set

In [20]:
from cnn import SuperConvNet
from solver import *
import time
from image_utils import *
import time


start_time = time.time()


# Prepare data: HERE X_train AND X_val ARE INVERTED

N = X_train.shape[0]
new_order = np.random.permutation(N) #np.random

data2 = data = {
    'X_train': X_val,
    'y_train': y_val,
    'X_val': X_train,
    'y_val': y_train
  }

print("\nData shape:")
for k, v in data2.iteritems():
  print '%s: ' % k, v.shape
print("\n")


# Parameters

Number_of_filters = 32
Filter_size = 7
Hidden_dimension = 100
Weight_scale = 0.001   # np.sqrt(2.0/N) ?
Regularization_strength = 0.00001
Number_of_epochs = 20
Batch_size = 10
Learning_rate = 1e-4



# Train the network

start_time = time.time()

model = SuperConvNet(input_dim=(3, 128, 128),
                     num_filters = Number_of_filters,
                     filter_size = Filter_size,
                     hidden_dim = Hidden_dimension,
                     num_classes = Nsubjects,
                     weight_scale = Weight_scale,
                     reg = Regularization_strength,
                     dtype = np.float32) #np.float32

solver = Solver(model, data,
                num_epochs = Number_of_epochs,
                batch_size = Batch_size,
                update_rule = 'adam',
                optim_config = {'learning_rate': Learning_rate,},
                verbose=True, print_every=1000)

solver.train()

t = time.time() - start_time
print("--- %s seconds ---" % t)



# Create report

comment="Training and val dataset inverted, no dropout"
write_report(Nsubjects,
             Number_of_filters,
             Filter_size, 
             Hidden_dimension, 
             Weight_scale, 
             Regularization_strength, 
             Learning_rate, 
             Number_of_epochs,
             Batch_size, 
             solver.train_acc_history[-1], 
             solver.val_acc_history[-1],
             solver.loss_history[-1], 
             t, 
             comment)

plt.subplot(2, 1, 1)
plt.plot(solver.loss_history, 'o')
plt.xlabel('iteration')
plt.ylabel('loss')

plt.subplot(2, 1, 2)
plt.plot(solver.train_acc_history, '-o')
plt.plot(solver.val_acc_history, '-o')
plt.legend(['train', 'val'], loc='upper left')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.show()

# Plot validation accuracy per subject

plt.subplot(2, 1, 2)
plt.plot(solver.val_acc['subject_1'], '-o')
plt.plot(solver.val_acc['subject_2'], '-o')
plt.plot(solver.val_acc['subject_3'], '-o')
plt.plot(solver.val_acc['subject_4'], '-o')
plt.plot(solver.val_acc['subject_5'], '-o')
plt.plot(solver.val_acc['subject_6'], '-o')
plt.legend(['1', '2', '3', '4', '5', '6'], loc='upper right')
plt.xlabel('epoch')
plt.ylabel('val accuracy')
plt.show()

# Plot datasets compostion:

print("Training set:"),
unique, counts = np.unique(y_val, return_counts=True) #np.unique
for i in range(unique.shape[0]):
    print '\t%d: %d' % (unique[i]+1, counts[i]),

print("\nValid set:"),
unique, counts = np.unique(y_train, return_counts=True) #np.unique
for i in range(unique.shape[0]):
    print '\t%d: %d' % (unique[i]+1, counts[i]),
print("\n")

ImportError: cannot import name SuperConvNet

### Try with Probes as training set and Gallery as validation set + Dropout

In [21]:
from cnn import ConvNetDrop
from solver import *
import time
from image_utils import *
import time


start_time = time.time()


# Prepare data: HERE X_train AND X_val ARE INVERTED

N = X_train.shape[0]
new_order = np.random.permutation(N) #np.random

data2 = data = {
    'X_train': X_val,
    'y_train': y_val,
    'X_val': X_train,
    'y_val': y_train
  }



# Parameters

Number_of_filters = 32
Filter_size = 7
Hidden_dimension = 100
Weight_scale = 0.001   # np.sqrt(2.0/N) ?
Regularization_strength = 0.00001
Number_of_epochs = 10
Batch_size = 10
Learning_rate = 1e-4


dropout = 0.2
seed = 123

# Train the network

start_time = time.time()

model = ConvNetDrop(input_dim=(3, 128, 128),
                     num_filters = Number_of_filters,
                     filter_size = Filter_size,
                     hidden_dim = Hidden_dimension,
                     num_classes = Nsubjects,
                     weight_scale = Weight_scale,
                     reg = Regularization_strength,
                     dtype = np.float32, #np.float32
                     dropout = dropout,
                     seed = seed)

solver = Solver(model, data,
                num_epochs = Number_of_epochs,
                batch_size = Batch_size,
                update_rule = 'adam',
                optim_config = {'learning_rate': Learning_rate,},
                verbose=True, print_every=20)

solver.train()

t = time.time() - start_time
print("--- %s seconds ---" % t)



# Create report

comment="Training and val dataset inverted, dropout=0.5"
write_report(Nsubjects,
             Number_of_filters,
             Filter_size, 
             Hidden_dimension, 
             Weight_scale, 
             Regularization_strength, 
             Learning_rate, 
             Number_of_epochs,
             Batch_size, 
             solver.train_acc_history[-1], 
             solver.val_acc_history[-1],
             solver.loss_history[-1], 
             t, 
             comment)

plt.subplot(2, 1, 1)
plt.plot(solver.loss_history, 'o')
plt.xlabel('iteration')
plt.ylabel('loss')

plt.subplot(2, 1, 2)
plt.plot(solver.train_acc_history, '-o')
plt.plot(solver.val_acc_history, '-o')
plt.legend(['train', 'val'], loc='upper left')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.show()

# Plot validation accuracy per subject

plt.subplot(2, 1, 2)
plt.plot(solver.val_acc['subject_1'], '-o')
plt.plot(solver.val_acc['subject_2'], '-o')
plt.plot(solver.val_acc['subject_3'], '-o')
plt.plot(solver.val_acc['subject_4'], '-o')
plt.plot(solver.val_acc['subject_5'], '-o')
plt.plot(solver.val_acc['subject_6'], '-o')
plt.legend(['1', '2', '3', '4', '5', '6'], loc='upper right')
plt.xlabel('epoch')
plt.ylabel('val accuracy')
plt.show()

# Plot datasets compostion:

print("Training set:"),
unique, counts = np.unique(y_val, return_counts=True) #np.unique
for i in range(unique.shape[0]):
    print '\t%d: %d' % (unique[i]+1, counts[i]),

print("\nValid set:"),
unique, counts = np.unique(y_train, return_counts=True) #np.unique
for i in range(unique.shape[0]):
    print '\t%d: %d' % (unique[i]+1, counts[i]),
print("\n")

NameError: name 'X_train' is not defined